In [1]:
# Secure Multi-Party Computation (SMPC) Lab Implementation
# Student: [Abdullah bin salman]
# Enrollment: [39]

import random
from sympy import symbols, expand, mod_inverse

print("=== Secure Multi-Party Computation (SMPC) Lab ===\n")

# =============================================================================
# PART I: MPC SUM USING SECRET SHARING
# =============================================================================

print("=== PART I: MPC Sum using Secret Sharing ===")

def generate_shares(secret, num_shares=3):
    """Generate random shares that sum to the secret"""
    shares = []
    for i in range(num_shares - 1):
        shares.append(random.randint(0, secret))
    shares.append(secret - sum(shares))
    random.shuffle(shares)
    return shares

# Private values for each party
abdullah_secret = 70
ali_secret = 80
mohammed_secret = 70

print("Private values:")
print(f"Abdullah: {abdullah_secret}")
print(f"Ali: {ali_secret}")
print(f"Mohammed: {mohammed_secret}")

# Generate shares for each party
abdullah_shares = generate_shares(abdullah_secret)
ali_shares = generate_shares(ali_secret)
mohammed_shares = generate_shares(mohammed_secret)

print(f"\nAbdullah shares: {abdullah_shares}")
print(f"Ali shares: {ali_shares}")
print(f"Mohammed shares: {mohammed_shares}")

# Distribute shares (each party gets one share from others)
abdullah_received = [abdullah_shares[0], ali_shares[0], mohammed_shares[0]]
ali_received = [abdullah_shares[1], ali_shares[1], mohammed_shares[1]]
mohammed_received = [abdullah_shares[2], ali_shares[2], mohammed_shares[2]]

print(f"\nReceived shares:")
print(f"Abdullah receives: {abdullah_received}")
print(f"Ali receives: {ali_received}")
print(f"Mohammed receives: {mohammed_received}")

# Each party computes partial sum
abdullah_partial = sum(abdullah_received)
ali_partial = sum(ali_received)
mohammed_partial = sum(mohammed_received)

print(f"\nPartial sums:")
print(f"Abdullah partial sum: {abdullah_partial}")
print(f"Ali partial sum: {ali_partial}")
print(f"Mohammed partial sum: {mohammed_partial}")

# Final sum
final_sum = abdullah_partial + ali_partial + mohammed_partial
print(f"\n✅ Final computed sum: {final_sum}")
print(f"✅ Actual sum: {abdullah_secret + ali_secret + mohammed_secret}")

# =============================================================================
# PART II: SHAMIR'S SECRET SHARING
# =============================================================================

print("\n=== PART II: Shamir's Secret Sharing ===")

class ShamirSecretSharing:
    def __init__(self, prime=97):
        self.prime = prime  # Prime number for finite field

    def generate_polynomial(self, secret, k, n):
        """Generate polynomial of degree k-1 with given secret"""
        coefficients = [secret] + [random.randint(1, self.prime-1) for _ in range(k-1)]
        return coefficients

    def generate_shares(self, coefficients, n):
        """Generate n shares from polynomial coefficients"""
        x = symbols('x')
        polynomial = sum(coef * x**i for i, coef in enumerate(coefficients))

        shares = []
        for i in range(1, n+1):
            y = expand(polynomial).subs(x, i) % self.prime
            shares.append((i, y))
        return shares

    def reconstruct_secret(self, shares):
        """Reconstruct secret using Lagrange interpolation"""
        k = len(shares)
        secret = 0

        for i in range(k):
            xi, yi = shares[i]
            numerator = 1
            denominator = 1

            for j in range(k):
                if i != j:
                    xj, _ = shares[j]
                    numerator = (numerator * (-xj)) % self.prime
                    denominator = (denominator * (xi - xj)) % self.prime

            lagrange_coef = (numerator * mod_inverse(denominator, self.prime)) % self.prime
            secret = (secret + yi * lagrange_coef) % self.prime

        return secret

# Test Shamir's Secret Sharing
sss = ShamirSecretSharing()

secret = 42
k = 3  # Minimum shares needed
n = 5  # Total shares to generate

print(f"Original secret: {secret}")
print(f"Scheme: ({k}, {n}) threshold")

# Generate polynomial and shares
coefficients = sss.generate_polynomial(secret, k, n)
shares = sss.generate_shares(coefficients, n)

print(f"Polynomial coefficients: {coefficients}")
print(f"Generated shares: {shares}")

# Reconstruct with k shares
selected_shares = shares[:k]  # Use first k shares
reconstructed_secret = sss.reconstruct_secret(selected_shares)

print(f"Shares used for reconstruction: {selected_shares}")
print(f"✅ Reconstructed secret: {reconstructed_secret}")

# Test with insufficient shares
try:
    insufficient_shares = shares[:2]  # Only 2 shares (less than k=3)
    failed_secret = sss.reconstruct_secret(insufficient_shares)
    print(f"❌ This should not print: {failed_secret}")
except:
    print("❌ Cannot reconstruct with insufficient shares (as expected)")

# =============================================================================
# PART III: GARBLED CIRCUITS (AND GATE)
# =============================================================================

print("\n=== PART III: Garbled Circuits - AND Gate ===")

class GarbledANDGate:
    def __init__(self):
        # Generate random labels for inputs and outputs
        self.labels = {
            'A0': random.getrandbits(64),
            'A1': random.getrandbits(64),
            'B0': random.getrandbits(64),
            'B1': random.getrandbits(64),
            'OUT0': random.getrandbits(64),
            'OUT1': random.getrandbits(64)
        }

        # Create garbled table
        self.garbled_table = self._create_garbled_table()

    def _encrypt(self, label1, label2, output_label):
        """Simple XOR-based encryption"""
        return label1 ^ label2 ^ output_label

    def _create_garbled_table(self):
        """Create garbled truth table"""
        table = []

        # Truth table for AND gate
        # A B | OUT
        # 0 0 | 0
        table.append(self._encrypt(self.labels['A0'], self.labels['B0'], self.labels['OUT0']))
        # 0 1 | 0
        table.append(self._encrypt(self.labels['A0'], self.labels['B1'], self.labels['OUT0']))
        # 1 0 | 0
        table.append(self._encrypt(self.labels['A1'], self.labels['B0'], self.labels['OUT0']))
        # 1 1 | 1
        table.append(self._encrypt(self.labels['A1'], self.labels['B1'], self.labels['OUT1']))

        random.shuffle(table)  # Shuffle to hide truth table
        return table

    def get_input_labels(self, party, value):
        """Get appropriate input labels for each party"""
        if party == 'omar':
            return self.labels[f'A{value}']
        elif party == 'salem':
            return self.labels[f'B{value}']
        else:
            raise ValueError("Invalid party name")

    def evaluate(self, labelA, labelB):
        """Evaluate garbled circuit with input labels"""
        for encrypted_output in self.garbled_table:
            possible_output = labelA ^ labelB ^ encrypted_output
            if possible_output in [self.labels['OUT0'], self.labels['OUT1']]:
                return 0 if possible_output == self.labels['OUT0'] else 1
        return None

# Test Garbled Circuit
print("Garbled AND Gate Implementation")

# Create garbled gate
gate = GarbledANDGate()

# Test all input combinations
test_cases = [(0, 0), (0, 1), (1, 0), (1, 1)]

print("\nTesting all input combinations:")
for a, b in test_cases:
    # Omar provides input A, Salem provides input B
    omar_label = gate.get_input_labels('omar', a)
    salem_label = gate.get_input_labels('salem', b)

    # Evaluate without knowing actual inputs
    result = gate.evaluate(omar_label, salem_label)

    print(f"Omar input: {a}, Salem input: {b} → Output: {result}")
    print(f"  Omar's label: {omar_label:016X}...")
    print(f"  Salem's label: {salem_label:016X}...")
    print(f"  Expected: {a and b}, Got: {result} ✅" if result == (a and b) else " ❌")

print("\n=== Lab Implementation Complete ===")














=== Secure Multi-Party Computation (SMPC) Lab ===

=== PART I: MPC Sum using Secret Sharing ===
Private values:
Abdullah: 70
Ali: 80
Mohammed: 70

Abdullah shares: [-29, 40, 59]
Ali shares: [4, 21, 55]
Mohammed shares: [57, 58, -45]

Received shares:
Abdullah receives: [-29, 4, 57]
Ali receives: [40, 21, 58]
Mohammed receives: [59, 55, -45]

Partial sums:
Abdullah partial sum: 32
Ali partial sum: 119
Mohammed partial sum: 69

✅ Final computed sum: 220
✅ Actual sum: 220

=== PART II: Shamir's Secret Sharing ===
Original secret: 42
Scheme: (3, 5) threshold
Polynomial coefficients: [42, 30, 50]
Generated shares: [(1, 25), (2, 11), (3, 0), (4, 89), (5, 84)]
Shares used for reconstruction: [(1, 25), (2, 11), (3, 0)]
✅ Reconstructed secret: 42
❌ This should not print: 39

=== PART III: Garbled Circuits - AND Gate ===
Garbled AND Gate Implementation

Testing all input combinations:
Omar input: 0, Salem input: 0 → Output: 0
  Omar's label: DE47700C72155C84...
  Salem's label: 0DE058783DF4B8F7.